In [3]:
import pandas as pd 

In [4]:
import os 

In [6]:
with open(r'../QED_data\QED-2-to-2-diag-TreeLevel-0.txt', 'r') as f:
    data = f.read()
    data2 = f.readlines()

In [12]:
with open(r'trying.txt' , 'w') as fp:
    fp.write(data) 

In [11]:
len(data)

1476753

In [ ]:

interaction_pattern = r'Interaction:\s*(.*?)\s*to\s*(.*?):'
vertex_pattern = r'Vertex V_1:(.*?), Vertex V_0:(.*?),'
# We are trying to search from one colon to another colon in case of amplitude_pattern, square 
# amplitude pattern is nothing but first colon from the right end of the line
amplitude_pattern = r':\s*(.*?)\s*:(?!.*:)'
squared_amplitude_pattern = r':\s*(.*?)\s*(?=\n|$)'


In [ ]:
import re
interactions = re.findall(interaction_pattern, data)
vertices = re.findall(vertex_pattern, data)
amplitudes = re.findall(amplitude_pattern, data)
squared_amplitudes = re.findall(squared_amplitude_pattern, data)

print(len(interactions), len(vertices), len(amplitudes), len(squared_amplitudes))


1728 1300 1728 1728


In [33]:
squared_amplitudes

['e_gam_239(X)^(*)  e_del_219(X)^(*)  to  e_eps_36(X)  e_eta_132(X) : Vertex V_1:e(X_2), e(X_4),  OffShell A(V_1), Vertex V_0:e(X_1), e(X_3),  OffShell A(V_0), : -1/2*i*e^2*gamma_{+%\\sigma_165,%gam_145,%gam_146}*gamma_{%\\sigma_165,%gam_147,%del_137}*e_{i_3,%gam_146}(p_1)_u*e_{k_3,%del_137}(p_2)_u*e_{l_3,%gam_145}(p_3)_u^(*)*e_{i_5,%gam_147}(p_4)_u^(*)/(m_e^2 + -s_13 + 1/2*reg_prop) : 2*e^4*(m_e^4 + -1/2*m_e^2*s_13 + 1/2*s_14*s_23 + -1/2*m_e^2*s_24 + 1/2*s_12*s_34)*(m_e^2 + -s_13 + 1/2*reg_prop)^(-2)',
 'e_gam_239(X)^(*)  e_del_219(X)^(*)  to  e_eps_36(X)  e_eta_132(X) : Vertex V_0:e(X_2), e(X_3),  OffShell A(V_0), Vertex V_1:e(X_1), e(X_4),  OffShell A(V_1), : 1/2*i*e^2*gamma_{+%\\sigma_172,%gam_162,%del_144}*gamma_{%\\sigma_172,%gam_163,%gam_164}*e_{i_3,%gam_164}(p_1)_u*e_{k_3,%del_144}(p_2)_u*e_{l_3,%gam_162}(p_3)_u^(*)*e_{i_5,%gam_163}(p_4)_u^(*)/(m_e^2 + -s_23 + 1/2*reg_prop) : 2*e^4*(m_e^4 + -1/2*m_e^2*s_14 + -1/2*m_e^2*s_23 + 1/2*s_13*s_24 + 1/2*s_12*s_34)*(m_e^2 + -s_23 + 1/2*

In [15]:
interactions

[('e_gam_239(X)^(*)  e_del_219(X)^(*)', 'e_eps_36(X)  e_eta_132(X) '),
 ('e_gam_239(X)^(*)  e_del_219(X)^(*)', 'e_eps_36(X)  e_eta_132(X) '),
 ('e_gam_239(X)^(*)  e_del_219(X)^(*)', 'e_eps_36(X)  OffShell e_del_353(X) '),
 ('e_gam_239(X)^(*)  e_del_219(X)^(*)', 'e_eps_36(X)  OffShell e_del_353(X) '),
 ('e_gam_239(X)^(*)  e_del_219(X)^(*)', 'OffShell e_eps_54(X)  e_eta_190(X) '),
 ('e_gam_239(X)^(*)  e_del_219(X)^(*)', 'OffShell e_eps_54(X)  e_eta_190(X) '),
 ('e_gam_239(X)^(*)  e_del_219(X)^(*)',
  'OffShell e_eps_54(X)  OffShell e_eta_216(X) '),
 ('e_gam_239(X)^(*)  e_del_219(X)^(*)',
  'OffShell e_eps_54(X)  OffShell e_eta_216(X) '),
 ('e_gam_239(X)^(*)  OffShell e_eta_248(X)^(*)',
  'e_alpha_241(X)  e_beta_49(X) '),
 ('e_gam_239(X)^(*)  OffShell e_eta_248(X)^(*)',
  'e_alpha_241(X)  e_beta_49(X) '),
 ('e_gam_239(X)^(*)  OffShell e_eta_248(X)^(*)',
  'e_alpha_241(X)  OffShell e_del_845(X) '),
 ('e_gam_239(X)^(*)  OffShell e_eta_248(X)^(*)',
  'e_alpha_241(X)  OffShell e_del_845(X) ')

In [34]:
def clean_text(text):
    if text:
        text = text.replace('\n', ' ').replace('  ', ' ')
        text = text.replace('AntiPart', 'AntiPart_')
        text = text.replace('OffShell', 'OffShell_')
        text = text.replace('(', '').replace(')', '')
        text = text.strip()
    return text


In [35]:
def split_terms(amplitude):
    qcd_terms = []
    qed_terms = []
    
    terms = amplitude.split('*')
    for term in terms:
        if 'd_{' in term:  # QCD term
            qcd_terms.append(term)
        elif 'e_{' in term:  # QED term
            qed_terms.append(term)
    
    return ' * '.join(qed_terms), ' * '.join(qcd_terms)


In [37]:
import pandas as pd

# Build the DataFrame
dataset = []
for i in range(len(interactions)):
    initial = interactions[i][0] if len(interactions[i]) > 0 else 'MISSING'
    final = interactions[i][1] if len(interactions[i]) > 1 else 'MISSING'
    vertex_1 = vertices[i][0] if i < len(vertices) and len(vertices[i]) > 0 else 'MISSING'
    vertex_0 = vertices[i][1] if i < len(vertices) and len(vertices[i]) > 1 else 'MISSING'
    amplitude = amplitudes[i] if i < len(amplitudes) else '0'
    squared_amplitude = squared_amplitudes[i] if i < len(squared_amplitudes) else '0'
    
    # If vertex_1 is missing but vertex_0 is present → Fill vertex_1 with vertex_0
    if vertex_1 == 'MISSING' and vertex_0 != 'MISSING':
        vertex_1 = vertex_0
    
    # If vertex_0 is missing but vertex_1 is present → Fill vertex_0 with vertex_1
    if vertex_0 == 'MISSING' and vertex_1 != 'MISSING':
        vertex_0 = vertex_1
    
    # If both are missing → Insert 'NO_VERTEX'
    if vertex_1 == 'MISSING' and vertex_0 == 'MISSING':
        vertex_1 = 'NO_VERTEX'
        vertex_0 = 'NO_VERTEX'

    dataset.append({
        'Initial Particles': initial.strip(),
        'Final Particles': final.strip(),
        'Vertex 1': vertex_1.strip(),
        'Vertex 2': vertex_0.strip(),
        'Amplitude': amplitude.strip(),
        'Squared Amplitude': squared_amplitude.strip()
    })

# Create DataFrame
df = pd.DataFrame(dataset)

# Preview DataFrame
df.head()


,Initial Particles,Final Particles,Vertex 1,Vertex 2,Amplitude,Squared Amplitude
0,e_gam_239(X)^(*) e_del_219(X)^(*),e_eps_36(X) e_eta_132(X),"e(X_2), e(X_4), OffShell A(V_1)",e(X_1),e_gam_239(X)^(*) e_del_219(X)^(*) to e_eps_...,e_gam_239(X)^(*) e_del_219(X)^(*) to e_eps_...
1,e_gam_239(X)^(*) e_del_219(X)^(*),e_eps_36(X) e_eta_132(X),"e(X_2), OffShell e(X_4), OffShell A(V_1)",e(X_1),e_gam_239(X)^(*) e_del_219(X)^(*) to e_eps_...,e_gam_239(X)^(*) e_del_219(X)^(*) to e_eps_...
2,e_gam_239(X)^(*) e_del_219(X)^(*),e_eps_36(X) OffShell e_del_353(X),"e(X_2), e(X_4), OffShell A(V_1)",e(X_1),e_gam_239(X)^(*) e_del_219(X)^(*) to e_eps_...,e_gam_239(X)^(*) e_del_219(X)^(*) to e_eps_...
3,e_gam_239(X)^(*) e_del_219(X)^(*),e_eps_36(X) OffShell e_del_353(X),"e(X_2), OffShell e(X_4), OffShell A(V_1)",e(X_1),e_gam_239(X)^(*) e_del_219(X)^(*) to e_eps_...,e_gam_239(X)^(*) e_del_219(X)^(*) to e_eps_...
4,e_gam_239(X)^(*) e_del_219(X)^(*),OffShell e_eps_54(X) e_eta_190(X),"OffShell e(X_2), e(X_4), OffShell A(V_1)",e(X_1),e_gam_239(X)^(*) e_del_219(X)^(*) to OffShe...,e_gam_239(X)^(*) e_del_219(X)^(*) to OffShe...


In [39]:
df.columns = ['Initial_particles', 'Final_particles', 'Vertex_1', 'Vertex_0', 'Amplitude', 'Squared_Amplitude']

Vertex_1
NO_VERTEX                                                                 428
AntiPart e(X_3), A(X_4),  OffShell e(V_1)                                  14
AntiPart  OffShell e(X_3), A(X_4),  OffShell e(V_1)                        14
AntiPart e(X_3),  OffShell A(X_4),  OffShell e(V_1)                        13
AntiPart  OffShell e(X_3),  OffShell A(X_4),  OffShell e(V_1)              13
                                                                         ... 
AntiPart  OffShell e(X_2),  OffShell A(X_3), AntiPart  OffShell e(V_1)      2
A(X_2),  OffShell e(X_3), AntiPart  OffShell e(V_1)                         2
AntiPart  OffShell e(X_1), A(X_3), AntiPart  OffShell e(V_1)                1
AntiPart  OffShell e(X_2), A(X_4), AntiPart  OffShell e(V_1)                1
AntiPart e(X_1), A(X_3), AntiPart  OffShell e(V_1)                          1
Name: count, Length: 193, dtype: int64